In [9]:
from dataclasses import dataclass
from pathlib import Path
import os

In [10]:
%pwd

'/Users/saiprakashlikky/Desktop/Projects/ML_projects/Flight_Fare_estimator_Project'

In [4]:
os.chdir("../")

In [5]:
%pwd

'/Users/saiprakashlikky/Desktop/Projects/ML_projects/Flight_Fare_estimator_Project'

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    datapath: Path

In [12]:
from src.Flight_Fare_estimator_Project.constants import *
from src.Flight_Fare_estimator_Project.utils.common import read_yaml, create_directories

In [75]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        datatransformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            datapath=config.datapath
        )

        return datatransformation_config


In [76]:
import warnings
import os
import urllib.request as request
import zipfile
from src.Flight_Fare_estimator_Project import logger
from src.Flight_Fare_estimator_Project.utils.common import get_size
import pandas as pd

In [77]:
import numpy as np 
import pickle
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler

In [78]:
class DataTransformation:
    def __init__(self, config):
        self.config = config

    def handle_missing_values(self):
        """
        Method: handle_missing_values
        Description: This method is used to handle missing values in Dataframe.
        Parameters: None
        Return: DataFrame after missing the dataset
        Version: 1.0
        """
        try:
            logger.info("Loading the Preprocessed DataFrame")
            # Redirect warnings to the logging system
            warnings.filterwarnings("ignore", category=UserWarning, module="pandas")
            self.df = pd.read_csv(self.config.datapath)
            logger.info(f"Our Data frame consists \n {self.df.isnull().sum()} so Here we can drop the na values ")
            self.df.dropna(inplace=True)
            logger.info("Dropped the nan values ")
            logger.info(f"Null values are handling na values is: {self.df.isnull().sum()}")  # Log the shape
            return self.df

        except Exception as e:
            raise e
        
    def outlier_treatment(self,df):
        """
        Method: Handling Outliers
        Description: This method is used to handle Outliers values in Dataframe.
        Parameters: dataframe
        Return: DataFrame after Handling outliers the dataset
        Version: 1.0
        """
        try:
            logger.info("Calaculating IQR to eliminate outliers from Price variable")
            Q1 = df.Price.quantile(0.25)
            Q3 = df.Price.quantile(0.75)
            IQR = Q3 - Q1
            logger.info(f"IQR for the Outlier treatment Vaiable Price is {IQR}")
            lower_bridge_ = Q1 - (IQR * 3)
            upper_bridge_ = Q3 + (IQR * 3)
            df.loc[df['Price'] > upper_bridge_, 'Price']
            logger.info("Handled the outliers of price variable")
            return df
        except Exception as e:
            raise e
        

    def feature_classification(self, df,file_path):
        """
        Method: classifying Dependent and Independent variables
        Description: This method is used to classify Dependent and Independent variables in Dataframe.
        Parameters: dataframe
        Return: dependent avraibles and Independent variables
        Version: 1.0
        """
        try:
            logger.info("Classification of Independent and Dependent variable")
            x = df.drop(columns=['Price'], axis=1)
            y = df['Price']
            logger.info(f"Dependent variables are {x.columns}, and Independent variables are {y.name}")
            return x, y
        except Exception as e:
            raise e



    def save_to_csv(self, df,file_path):
        """
        Method: save_to_csv
        Description: This method is used to save the DataFrame to a CSV file.
        Parameters:
            - df: DataFrame
            - file_path: Path to save the CSV file
        Return: None
        Version: 1.0
        """
        try:
            output_filepath = os.path.join(self.config.root_dir, file_path)
            if not os.path.exists(output_filepath):
                logger.info(f"Saving DataFrame to CSV file: {output_filepath}")
                df.to_csv(output_filepath, index=False)
                logger.info("DataFrame saved successfully.")
            else:
                logger.warning(f"CSV file already exists at {output_filepath}. Not saving.")
        except Exception as e:
            raise e

In [79]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    dataframe=data_transformation.handle_missing_values()
    dataframe=data_transformation.outlier_treatment(dataframe)
    x,y=data_transformation.feature_classification(dataframe,'target_feature.csv')
    data_transformation.save_to_csv(x, 'x_output.csv')
    data_transformation.save_to_csv(y, 'y_output.csv')         
except Exception as e:
    raise e


[2023-11-27 23:36:26,341:INFO:common:yaml file: config/config.yaml loaded successfully]
[2023-11-27 23:36:26,343:INFO:common:yaml file: params.yaml loaded successfully]
[2023-11-27 23:36:26,344:INFO:common:yaml file: schema.yaml loaded successfully]
[2023-11-27 23:36:26,344:INFO:common:created directory at: artifacts]
[2023-11-27 23:36:26,345:INFO:common:created directory at: artifacts/data_transformation]
[2023-11-27 23:36:26,345:INFO:2658766695:Loading the Preprocessed DataFrame]
[2023-11-27 23:36:26,368:INFO:2658766695:Our Data frame consists 
 Airline                      0
Source                       0
Destination                  0
Total_Stops                  1
Price                        0
Month_of_Month_of_journey    0
day_of_date_of_journey       0
Duration_minutes             0
dtype: int64 so Here we can drop the na values ]
[2023-11-27 23:36:26,372:INFO:2658766695:Dropped the nan values ]
[2023-11-27 23:36:26,374:INFO:2658766695:Null values are handling na values is: Air